In [130]:
#using pandas DataFrame to hold training, testing, and validation data
import pandas as pd
from pandas import set_option
set_option("display.max_rows", 10)

LARGE_FIGSIZE = (12, 8)
import math

names_of_attributes = names=["label","cap-shape","cap-surface","cap-color","bruises","oder",
                                "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                                "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                                "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                                "veil-color","ring-number","ring-type","spore-print-color",
                                "population","habitat"]

#read training data into pandas DataFrame
training_data = pd.read_table("data/training.txt", sep=",", names=names_of_attributes)

#read testing data into pandas DataFrame
testing_data = pd.read_table("data/testing.txt", sep=",", names=names_of_attributes)


#represents a decision node in the tree
class Node:
    #identifies which attribute this node tests on
    decision_attribute = ""
    
    #initialize node
    def __init__(self):
        self.decision_attribute = ""
        self.branches = {}
        
    #adds a child Node or Leaf 
    def add_branch(self,attribute_value,node):
        self.branches[attribute_value] = node
    
    #returns all branches
    def get_branches(self):
        return self.branches

#represents a leaf node in the tree
class Leaf:
    #label representing the classification value (e or p)
    label = ""

#global variables for the positive and negative label values (e and p)    
positive_value = "e"
negative_value = "p"

#global variable for setting the evaluation criteria (entropy or misclassification)
#can be set just before calling the id3 algorithm below
evaluation_criteria = "entropy"

#look-up table for the chi-square values (source https://dl.dropboxusercontent.com/u/63267778/chi_square_table.txt)
#key = dof, values[0] = alpha 0.5, values[1] =  alpha 0.05, values[2] = alpha 0.01
chi_square_look_up = {
                    1: [0.4549,3.841,6.635],
                    2: [1.3863,5.991,9.21],
                    3: [2.366,7.815,11.345],
                    4: [3.3567,9.488,13.277],
                    5: [4.3515,11.07,15.086],
                    6: [5.3481,12.592,16.812],
                    7: [6.3458,14.067,18.475],
                    8: [7.3441,15.507,20.09],
                    9: [8.3428,16.919,21.666],
                    10: [9.3418,18.307,23.209],
                    11: [10.341,19.675,24.725],
                    12: [11.3403,21.026,26.217],
                    13: [12.3398,22.362,27.688],
                    14: [13.3393,23.685,29.141],
                    15: [14.3389,24.996,30.578],
                    16: [15.3385,26.296,32],
                    17: [16.3382,27.587,33.409],
                    18: [17.3379,28.869,34.805],
                    19: [18.3377,30.144,36.191],
                    20: [19.3374,31.41,37.566]
                    }

#look-up table for the valid values of each attribute
valid_values = {"label": ["p","e"],
                "cap-shape": ["b","c","x","f","k","s"],
                "cap-surface": ["f","g","y","s"],
                "cap-color": ["n","b","c","g","r","p","u","e","w","y"],
                "bruises": ["t","f"],
                "oder": ["a","l","c","y","f","m","n","p","s"],
                "gill-attachment": ["a","d","f","n"],
                "gill-spacing": ["c","w","d"],
                "gill-size": ["b","n"],
                "gill-color": ["k","n","b","h","g","r","o","p","u","e","w","y"],
                "stalk-shape": ["e","t"],
                "stalk-root": ["b","c","u","e","z","r","?"],
                "stalk-surface-above-ring": ["f","y","k","s"],
                "stalk-surface-below-ring": ["f","y","k","s"],
                "stalk-color-above-ring": ["n","b","c","g","o","p","e","w","y"],
                "stalk-color-below-ring": ["n","b","c","g","o","p","e","w","y"],
                "veil-type": ["p","u"],
                "veil-color": ["n","o","w","y"],
                "ring-number": ["n","o","t"],
                "ring-type": ["c","e","f","l","n","p","s","z"],
                "spore-print-color": ["k","n","b","h","r","o","u","w","y"],
                "population": ["a","c","n","s","v","y"],
                "habitat": ["g","l","m","p","u","w","d"]
                }

#calculates the degrees of freedom for an attribute
def degress_of_freedom(attribute):
    return len(valid_values[attribute]) - 1

#returns the chi-square value given an attribute and alpha value
def chi_square_value_for_attribute(attribute,alpha):
    values = chi_square_look_up[degress_of_freedom(attribute)]
    if alpha == '0.5':
        return values[0]
    elif alpha == '0.05':
        return values[1]
    elif alpha == '0.01':
        return values[2]
    else:
        return 0.0
    
#returns the size (number of rows) of a DataFrame
def size(data):
    return len(data.index)

#returns a subset of a DataFrame, filtering on the value of an attribute
def filter_data(examples,attribute, value):
    return examples[examples[attribute] == value]
    
#tests if all the elements in the DataFrame (examples) have same value for an attribute    
def all_one_value(examples, attribute, attribute_value):
    filtered_examples = filter_data(examples, attribute, attribute_value)
    return size(filtered_examples) == size(examples)

#returns all possible values for an attribute
def possible_range_of_values(attribute):
    return valid_values[attribute]

#returns the most common value present the the examples for a given attribute
def extract_most_common_value(examples, attribute):
    values = possible_range_of_values(attribute)
    most_common = ""
    highest_count = 0
    for value in values:
        filtered = filter_data(examples, attribute, value)
        size_filtered = size(filtered)
        if size_filtered > highest_count:
            highest_count = size_filtered
            most_common = value  
    return most_common

#sub calculation for chi-square, calculates the chi-square for an attribute value
def chi_square_calculation_attribute_v(examples_v, target_attribute,attribute_v):
    total_count = size(examples_v)
    #if the size of examples_v is zero then there is nothing to calculate and return 0
    if total_count == 0:
        return 0.0
    
    positive_filtered_examples = filter_data(examples_v, target_attribute, positive_value)
    negative_filtered_examples = filter_data(examples_v, target_attribute, negative_value)
    
    observed_positive = size(positive_filtered_examples)
    expected_positive = (total_count/2.0)
                
    observed_negative = size(negative_filtered_examples)
    expected_negative = (total_count/2.0)
    
    pos = ((observed_positive - expected_positive)**2)/expected_positive
    neg = ((observed_negative - expected_negative)**2)/expected_negative
    return pos + neg
          
#returns the chi-square value for an attribute    
def chi_square_calculation(examples, target_attribute,attribute):
    chi_square_sum = 0.0
    values = possible_range_of_values(attribute)
    for value in values:
        examples_v = filter_data(examples, attribute,value)
        chi_square_v = chi_square_calculation_attribute_v(examples_v,target_attribute,value)
        chi_square_sum = chi_square_sum + chi_square_v
    return chi_square_sum
    
#calculates the entropy on a subset (examples_v) of data
def calculate_entropy(examples_v,target_attribute):
    entropy = 0.0
    total_size = size(examples_v)
    #if there is no data just return a zero value (does not contribute to calculation)
    if total_size == 0:
        return 0.0
    positive_filtered_examples_v = filter_data(examples_v, target_attribute, positive_value)
    negative_filtered_examples_v = filter_data(examples_v, target_attribute, negative_value)
    
    num_positive = size(positive_filtered_examples_v)
    calc_positive = 0.0
    if num_positive != 0:
        calc_positive = -(num_positive/total_size)*math.log((num_positive/total_size),2)
            
        
    num_negative = size(negative_filtered_examples_v)
    calc_negative = 0.0
    if num_negative != 0:
        calc_negative = - (num_negative/total_size)*math.log((num_negative/total_size),2)
        
    #returns -p(+)log p(+) - p(-)log p(-)    
    return calc_positive + calc_negative
   
#calculates the misclassification error on a subset (examples_v) of data
def calculate_misclassification_error(examples_v, target_attribute):
    error = 0.0
    total_size = size(examples_v)
    #if there is no data just return a zero value (does not contribute to calculation)
    if total_size == 0:
        return 0.0

    positive_filtered_examples_v = filter_data(examples_v,target_attribute,positive_value)
    negative_filtered_examples_v = filter_data(examples_v,target_attribute,negative_value)
    
    num_positive = size(positive_filtered_examples_v)
    p_positive = (num_positive/total_size)
                
    num_negative = size(negative_filtered_examples_v)
    p_negative = (num_negative/total_size)
    
    error = 1.0 - max([p_positive,p_negative])
    #returns 1 - max(probability of positive,probability of negative)
    return error
    
#returns the attribute with the best informatin gain using entropy as the impurity measure
def determine_best_attribute_entropy(examples, target_attribute, attributes):
    best_attribute = ""
    max_information_gain = 0.0
   
    #find the entropy of S (examples)
    entropy_s = calculate_entropy(examples,target_attribute)
    size_of_s = size(examples)
    
    
    #loop through attributes and calculate the gain
    for attribute in attributes:
        values = possible_range_of_values(attribute)
        information_gain = entropy_s
        for value in values:
            examples_v = filter_data(examples,attribute,value)
            entropy_v = calculate_entropy(examples_v,target_attribute)
            size_of_examples_v = size(examples_v)
            weight = (size_of_examples_v/size_of_s)
            information_gain = information_gain - weight*entropy_v
        #if we have a new candidate for the best attribute store the gain and attribute name
        if information_gain > max_information_gain:
            max_information_gain = information_gain
            best_attribute = attribute
             
    return best_attribute

#returns the attribute with the best gain using misclassification error as the impurity measure
def determine_best_attribute_misclassification_error(examples, target_attribute, attributes):
    best_attribute = ""
    max_gain = 0.0
    #find the misclassification error of S (examples)
    error_s = calculate_misclassification_error(examples,target_attribute)
    size_of_s = size(examples)
    
    #loop through attributes and calculate the gain
    for attribute in attributes:
        values = possible_range_of_values(attribute)
        gain = error_s
        for value in values:
            examples_v = filter_data(examples,attribute,value)
            error_v = calculate_misclassification_error(examples_v,target_attribute)
            size_of_examples_v = size(examples_v)
            weight = (size_of_examples_v/size_of_s)
            gain = gain - weight*error_v
        #if we have a new candidate for the best attribute store the gain and attribute name
        if gain > max_gain:
            max_gain = gain
            best_attribute = attribute
       
    return best_attribute

#performs the id3 decision tree algorithm (recursive), returns the root node of the tree
#  examples - set (pandas DataFrame) of training examples provided for current iteration of the algorithm
#  target_attribute - the name of the column that provides the classification label
#  attribute_list - set of the names of the attributes for the training examples (excludes target_attribute)
#  alpha - the alpha value ["0.01","0.05","0.5","1.0"] used for chi-square pruning 
def id3(examples, target_attribute, attribute_list, alpha):
    root = Node()
    if(all_one_value(examples, target_attribute, positive_value)):
        leaf = Leaf()
        leaf.label = positive_value
        return leaf
    if(all_one_value(examples, target_attribute, negative_value)):
        leaf = Leaf()
        leaf.label = negative_value
        return leaf
    if(len(attribute_list) == 0):
        leaf = Leaf()
        leaf.label = extract_most_common_value(examples,target_attribute)
        return leaf
    #evaluate node selection using entropy or misclassification error
    if evaluation_criteria == 'entropy':
        a = determine_best_attribute_entropy(examples, target_attribute, attribute_list)
    else:
        a = determine_best_attribute_misclassification_error(examples, target_attribute, attribute_list)
    
    #for pruning get the calculated and threshold chi-square value
    chi_square_calculated = chi_square_calculation(examples,target_attribute,a)
    chi_square_lookup = chi_square_value_for_attribute(a, alpha)
    
    #if the calculated value is less than the threshold value then prune (decision node -> leaf node)
    if chi_square_calculated < chi_square_lookup:
        leaf = Leaf()
        leaf.label = extract_most_common_value(examples,target_attribute)
        return leaf
    root.decision_attribute = a
   
    values = possible_range_of_values(a)
    #iterate over all the possible values of an attribute and create branches for each value
    for value in values:
        root.branches[value] = ""
        examples_v = filter_data(examples,a,value)
        size_of_examples_v = size(examples_v)
        #if there are no more examples to train on create a leaf node for this branch
        if size_of_examples_v == 0:
            leaf = Leaf()
            leaf.label = extract_most_common_value(examples,target_attribute)
            root.add_branch(value,leaf)
        else:
            #remove attribute a so we don't reprocess it
            if a in attribute_list:
                attribute_list.remove(a)
            #add the next best decision node or leaf
            root.add_branch(value, id3(examples_v,target_attribute,attribute_list, alpha))
    return root



#traverses the decision tree one level and returns the node or leaf at the end of the selected branch
#  example_attribute_values - the example's attribute values <v1,v2,...vn>
#  node - decision node
def split_child(example_attribute_values, node):
    #from the example get the value of the attribute associated with is node example
    example_decision_attribute_value = example_attribute_values[node.decision_attribute]
    #return the node or leaf of the branch associated with the example's value of the decision attribute 
    return node.get_branches()[example_decision_attribute_value]

#classify an example, return the value of the first leaf node discovered
#  example_attribute_values - the example's attribute values <v1,v2,...vn>
#  node - decision node
def classify(example_attribute_values, node):
    if type(node) is Leaf:
        return node.label
    else:
        return classify(example_attribute_values, split_child(example_attribute_values,node))
    
#returns the accuracy of classifing on a testing data set
#  testing_data - the data set to classify (pandas DataFrame)
#  id3_root - the root of the id3 decision tree
#  print_statement - the string to print
def calculate_accuracy(testing_data, id3_root, print_statement):
    count_correct = 0
    count_false = 0
    accuracy = 0.0
    for i,r in testing_data.iterrows():
        #if the classifing label in the data set matches the calculated one we classified correctly
        if r["label"] == classify(r,id3_root):
            count_correct = count_correct + 1
        else:
            count_false = count_false + 1
    if count_correct > 0:
        accuracy = (count_correct)/(count_correct + count_false)  * 100
    
    print(print_statement, accuracy, "%")
    print("  ", "Number correctly classified:   ", count_correct) 
    print("  ", "Number incorrectly classified: ", count_false) 
    return accuracy
    

#helper function for getting the list of attributes for the data sets
def get_attributes():
    
    #list of attributes to be used in the id3 algorithm. 
    #After an attribute is selected for a node of the tree it is removed from the list
    attributes = ["cap-shape","cap-surface","cap-color","bruises","oder",
                "gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape",
                "stalk-root","stalk-surface-above-ring","stalk-surface-below-ring",
                "stalk-color-above-ring","stalk-color-below-ring","veil-type",
                "veil-color","ring-number","ring-type","spore-print-color",
                "population","habitat"]
    return attributes

best_accuracy = 0.0
best_tree = Node()
best_tree_name = ''

#the following group of statements run the id3 algorithm for different values of evaluation criteria and alpha (for pruning)
evaluation_criteria = "entropy"
alpha = '1.0'
print_statement = "Accuracy (using entropy and confidence level of 0):  "
id3_entropy = id3(training_data,"label",get_attributes(),alpha)
accuracy = id3_entropy_accuracy = calculate_accuracy(testing_data,id3_entropy, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_tree = id3_entropy
    best_tree_name = 'id3_entropy_confidence_level_0'

evaluation_criteria = "entropy"
alpha = '0.01'
print_statement = "Accuracy (using entropy and confidence level of 99): "
id3_entropy_alpha_01 = id3(training_data,"label",get_attributes(),alpha)
accuracy = id3_entropy_alpha_01_accuracy= calculate_accuracy(testing_data,id3_entropy_alpha_01, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_tree = id3_entropy_alpha_01
    best_tree_name = 'id3_entropy_confidence_level_99'

evaluation_criteria = "entropy"
alpha = '0.05'
print_statement = "Accuracy (using entropy and confidence level of 95): "
id3_entropy_alpha_05 = id3(training_data,"label",get_attributes(),alpha)
accuracy = id3_entropy_alpha_05_accuracy = calculate_accuracy(testing_data,id3_entropy_alpha_05, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_tree = id3_entropy_alpha_05
    best_tree_name = 'id3_entropy_confidence_level_95'

evaluation_criteria = "entropy"
alpha = '0.5'
print_statement = "Accuracy (using entropy and confidence level of 50):  "
id3_entropy_alpha_5 = id3(training_data,"label",get_attributes(),alpha)
accuracy = id3_entropy_alpha_5_accuracy = calculate_accuracy(testing_data,id3_entropy_alpha_5, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_tree = id3_entropy_alpha_5
    best_tree_name = 'id3_entropy_confidence_level_50'


print("")

evaluation_criteria = "misclassification"
alpha = '1.0'
print_statement = "Accuracy (using misclassification and confidence level of 0):  "
id3_misclassification_error = id3(training_data,"label",get_attributes(),alpha)
accuracy = calculate_accuracy(testing_data,id3_misclassification_error, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    print(accuracy, best_accuracy)
    best_tree = id3_misclassification_error
    best_tree_name = 'id3_misclassification_error_confidence_level_0'

evaluation_criteria = "misclassification"
alpha = '0.01'
print_statement = "Accuracy (using misclassification and confidence level of 99): "
id3_misclassification_error_alpha01 = id3(training_data,"label",get_attributes(),alpha)
accuracy = calculate_accuracy(testing_data,id3_misclassification_error_alpha01, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_tree = id3_misclassification_error_alpha01
    best_tree_name = 'id3_misclassification_error_confidence_level_99'

evaluation_criteria = "misclassification"
alpha = '0.05'
print_statement = "Accuracy (using misclassification and confidence level of 95): "
id3_misclassification_error_alpha05 = id3(training_data,"label",get_attributes(),alpha)
accuracy = calculate_accuracy(testing_data,id3_misclassification_error_alpha05, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_tree = id3_misclassification_error_alpha05
    best_tree_name = 'id3_misclassification_error_confidence_level_95'

evaluation_criteria = "misclassification"
alpha = '0.5'
print_statement = "Accuracy (using misclassification and confidence level of 50):  "
id3_misclassification_error_alpha5 = id3(training_data,"label",get_attributes(),alpha)
accuracy = calculate_accuracy(testing_data,id3_misclassification_error_alpha5, print_statement)
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_tree = id3_misclassification_error_alpha5
    best_tree_name = 'id3_misclassification_error_confidence_level_50'
    

print("")    
print("best tree: ", best_tree_name)

print("")



#read validation data into pandas DataFrame    
validation_data = pd.read_table("data/validation.txt", sep=",", names=names_of_attributes)


import os
remove_chars = len(os.linesep)


file = open("validation-best-accuracy.txt", "w")
for i,r in validation_data.iterrows():
    file.write(classify(r,best_tree) + "\n")

#remove the last '\n', need for the online validation script to run correctly
file.truncate(file.tell() - remove_chars)
file.close()

print("classification predictions on validation data written to: validation-best-accuracy.txt")

print("")


Accuracy (using entropy and confidence level of 0):   100.0 %
   Number correctly classified:    2031
   Number incorrectly classified:  0
Accuracy (using entropy and confidence level of 99):  100.0 %
   Number correctly classified:    2031
   Number incorrectly classified:  0
Accuracy (using entropy and confidence level of 95):  100.0 %
   Number correctly classified:    2031
   Number incorrectly classified:  0
Accuracy (using entropy and confidence level of 50):   100.0 %
   Number correctly classified:    2031
   Number incorrectly classified:  0

Accuracy (using misclassification and confidence level of 0):   100.0 %
   Number correctly classified:    2031
   Number incorrectly classified:  0
Accuracy (using misclassification and confidence level of 99):  100.0 %
   Number correctly classified:    2031
   Number incorrectly classified:  0
Accuracy (using misclassification and confidence level of 95):  100.0 %
   Number correctly classified:    2031
   Number incorrectly classified

In [67]:
bright = training_data[training_data['cap-color'].isin(['e','y'])]


In [68]:
bright_p = bright[bright['label'] == 'p']
bright_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,y,e,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,l
22,p,f,y,e,f,y,f,c,n,b,...,k,p,p,p,w,o,e,w,v,l
24,p,x,y,y,f,f,f,c,b,p,...,k,p,b,p,w,o,l,h,y,p
31,p,k,y,e,f,s,f,c,n,b,...,k,w,w,p,w,o,e,w,v,d
38,p,x,y,y,f,f,f,c,b,p,...,k,p,p,p,w,o,l,h,v,p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,p,f,y,e,f,f,f,c,n,b,...,k,p,p,p,w,o,e,w,v,l
4046,p,x,f,y,f,f,f,c,b,h,...,k,n,n,p,w,o,l,h,y,d
4055,p,f,s,e,f,f,f,c,n,b,...,k,w,w,p,w,o,e,w,v,d
4056,p,k,s,e,f,y,f,c,n,b,...,s,w,w,p,w,o,e,w,v,p


In [66]:
bright_e = bright[bright['label'] == 'e']
bright_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
11,e,f,f,e,t,n,f,c,b,u,...,s,g,g,p,w,o,p,k,v,d
12,e,f,f,e,t,n,f,c,b,w,...,s,w,p,p,w,o,p,n,v,d
35,e,f,y,e,t,n,f,c,b,w,...,s,p,w,p,w,o,p,n,y,d
36,e,x,y,e,t,n,f,c,b,p,...,s,w,g,p,w,o,p,k,y,d
37,e,x,y,y,t,l,f,c,b,w,...,s,w,w,p,w,o,p,k,n,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3993,e,f,f,e,t,n,f,c,b,u,...,s,p,g,p,w,o,p,n,v,d
3999,e,x,y,e,t,n,f,c,b,w,...,s,g,p,p,w,o,p,n,y,d
4000,e,x,f,e,t,n,f,c,b,p,...,s,p,g,p,w,o,p,k,v,d
4043,e,f,y,e,t,n,f,c,b,n,...,s,p,g,p,w,o,p,n,y,d


In [69]:
smell_bad = training_data[training_data['oder'].isin(['y','f','m','p','s'])]

In [71]:
smell_bad_p = smell_bad[smell_bad['label'] == 'p']
smell_bad_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,y,e,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,l
2,p,f,s,n,t,p,f,c,n,p,...,s,w,w,p,w,o,p,n,s,u
3,p,x,y,g,f,f,f,c,b,g,...,k,b,n,p,w,o,l,h,y,p
14,p,x,y,n,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,v,u
16,p,k,y,n,f,s,f,c,n,b,...,k,p,p,p,w,o,e,w,v,l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4055,p,f,s,e,f,f,f,c,n,b,...,k,w,w,p,w,o,e,w,v,d
4056,p,k,s,e,f,y,f,c,n,b,...,s,w,w,p,w,o,e,w,v,p
4057,p,f,y,n,f,f,f,c,n,b,...,s,w,w,p,w,o,e,w,v,l
4058,p,x,y,y,f,f,f,c,b,h,...,k,p,n,p,w,o,l,h,y,p


In [72]:
smell_bad_e = smell_bad[smell_bad['label'] == 'e']
smell_bad_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat


In [101]:
pointed_or_umbrella_shaped = training_data[training_data['cap-shape'].isin(['c','x','b','k'])]

In [102]:
pointed_or_umbrella_shaped_p = pointed_or_umbrella_shaped[pointed_or_umbrella_shaped['label'] == 'p']
pointed_or_umbrella_shaped_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,y,e,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,l
3,p,x,y,g,f,f,f,c,b,g,...,k,b,n,p,w,o,l,h,y,p
14,p,x,y,n,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,v,u
16,p,k,y,n,f,s,f,c,n,b,...,k,p,p,p,w,o,e,w,v,l
24,p,x,y,y,f,f,f,c,b,p,...,k,p,b,p,w,o,l,h,y,p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4046,p,x,f,y,f,f,f,c,b,h,...,k,n,n,p,w,o,l,h,y,d
4051,p,k,y,n,f,s,f,c,n,b,...,k,w,w,p,w,o,e,w,v,d
4056,p,k,s,e,f,y,f,c,n,b,...,s,w,w,p,w,o,e,w,v,p
4058,p,x,y,y,f,f,f,c,b,h,...,k,p,n,p,w,o,l,h,y,p


In [103]:
pointed_or_umbrella_shaped_e = pointed_or_umbrella_shaped[pointed_or_umbrella_shaped['label'] == 'e']
pointed_or_umbrella_shaped_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
1,e,x,f,g,f,n,f,c,n,n,...,s,w,w,p,w,o,p,k,v,u
4,e,x,y,g,t,n,f,c,b,n,...,s,p,g,p,w,o,p,k,y,d
5,e,x,f,w,t,a,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
6,e,b,f,g,f,n,f,w,b,p,...,k,w,w,p,w,t,p,w,s,g
7,e,x,f,n,t,n,f,c,b,n,...,s,p,p,p,w,o,p,n,v,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,e,k,y,p,t,n,f,c,b,w,...,s,w,w,p,w,t,e,w,c,w
4047,e,x,y,g,t,n,f,c,b,n,...,s,g,g,p,w,o,p,k,v,d
4048,e,x,f,c,f,n,f,w,n,w,...,s,w,n,p,w,o,e,w,v,l
4052,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m


In [104]:
flat_cap = training_data[training_data['cap-shape'].isin(['f'])]

In [105]:
flat_cap_p = flat_cap[flat_cap['label'] == 'p']
flat_cap_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
2,p,f,s,n,t,p,f,c,n,p,...,s,w,w,p,w,o,p,n,s,u
19,p,f,y,n,f,f,f,c,n,b,...,k,p,w,p,w,o,e,w,v,l
22,p,f,y,e,f,y,f,c,n,b,...,k,p,p,p,w,o,e,w,v,l
25,p,f,y,g,f,f,f,c,b,h,...,k,n,b,p,w,o,l,h,v,p
26,p,f,f,g,f,f,f,c,b,g,...,k,n,n,p,w,o,l,h,v,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,p,f,f,y,f,f,f,c,b,h,...,k,p,p,p,w,o,l,h,y,d
4044,p,f,y,e,f,f,f,c,n,b,...,k,p,p,p,w,o,e,w,v,l
4050,p,f,y,g,f,f,f,c,b,h,...,k,b,b,p,w,o,l,h,y,d
4055,p,f,s,e,f,f,f,c,n,b,...,k,w,w,p,w,o,e,w,v,d


In [106]:
flat_cap_e = flat_cap[flat_cap['label'] == 'e']
flat_cap_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
10,e,f,y,g,t,n,f,c,b,w,...,s,g,w,p,w,o,p,n,y,d
11,e,f,f,e,t,n,f,c,b,u,...,s,g,g,p,w,o,p,k,v,d
12,e,f,f,e,t,n,f,c,b,w,...,s,w,p,p,w,o,p,n,v,d
13,e,f,s,w,f,n,f,w,b,k,...,f,w,w,p,w,o,e,k,s,g
17,e,f,y,r,f,n,f,c,n,h,...,f,w,w,p,w,o,f,h,v,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,e,f,f,e,t,n,f,c,b,u,...,s,g,w,p,w,o,p,n,v,d
4049,e,f,y,g,t,n,f,c,b,u,...,s,w,w,p,w,o,p,n,v,d
4053,e,f,f,w,f,n,f,w,b,p,...,s,w,w,p,w,o,e,k,a,g
4054,e,f,s,w,f,n,f,w,b,p,...,f,w,w,p,w,o,e,k,a,g


In [110]:
warts_or_scales = training_data[training_data['cap-surface'].isin(['s'])]

In [111]:
warts_or_scales_p = warts_or_scales[warts_or_scales['label'] == 'p']
warts_or_scales_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
2,p,f,s,n,t,p,f,c,n,p,...,s,w,w,p,w,o,p,n,s,u
29,p,x,s,g,t,f,f,c,b,h,...,f,w,w,p,w,o,p,h,v,g
55,p,f,s,n,f,s,f,c,n,b,...,k,p,p,p,w,o,e,w,v,p
59,p,k,s,e,f,s,f,c,n,b,...,k,p,w,p,w,o,e,w,v,p
65,p,x,s,b,t,f,f,c,b,p,...,s,w,w,p,w,o,p,h,s,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,p,f,s,b,t,f,f,c,b,h,...,s,w,w,p,w,o,p,h,s,u
4032,p,x,s,e,f,y,f,c,n,b,...,s,w,p,p,w,o,e,w,v,d
4034,p,f,s,n,f,s,f,c,n,b,...,s,p,p,p,w,o,e,w,v,l
4055,p,f,s,e,f,f,f,c,n,b,...,k,w,w,p,w,o,e,w,v,d


In [112]:
warts_or_scales_e = warts_or_scales[warts_or_scales['label'] == 'e']
warts_or_scales_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
13,e,f,s,w,f,n,f,w,b,k,...,f,w,w,p,w,o,e,k,s,g
18,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
23,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,n,o,p,b,v,l
27,e,x,s,g,f,n,f,w,b,k,...,f,w,w,p,w,o,e,k,s,g
28,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,c,l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,e,f,s,w,t,a,f,w,n,w,...,s,w,w,p,w,o,p,u,v,d
4019,e,b,s,w,t,l,f,c,b,w,...,s,w,w,p,w,o,p,k,s,m
4027,e,f,s,n,f,n,f,w,b,h,...,f,w,w,p,w,o,e,n,s,g
4054,e,f,s,w,f,n,f,w,b,p,...,f,w,w,p,w,o,e,k,a,g


In [113]:
bulbous_cup_or_sac = training_data[training_data['stalk-root'].isin(['b'])]

In [114]:
bulbous_cup_or_sac_p = bulbous_cup_or_sac[bulbous_cup_or_sac['label'] == 'p']
bulbous_cup_or_sac_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
3,p,x,y,g,f,f,f,c,b,g,...,k,b,n,p,w,o,l,h,y,p
24,p,x,y,y,f,f,f,c,b,p,...,k,p,b,p,w,o,l,h,y,p
25,p,f,y,g,f,f,f,c,b,h,...,k,n,b,p,w,o,l,h,v,p
26,p,f,f,g,f,f,f,c,b,g,...,k,n,n,p,w,o,l,h,v,g
29,p,x,s,g,t,f,f,c,b,h,...,f,w,w,p,w,o,p,h,v,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4040,p,x,f,g,f,f,f,c,b,h,...,k,b,p,p,w,o,l,h,v,g
4041,p,f,f,y,f,f,f,c,b,h,...,k,p,p,p,w,o,l,h,y,d
4046,p,x,f,y,f,f,f,c,b,h,...,k,n,n,p,w,o,l,h,y,d
4050,p,f,y,g,f,f,f,c,b,h,...,k,b,b,p,w,o,l,h,y,d


In [115]:
bulbous_cup_or_sac_e = bulbous_cup_or_sac[bulbous_cup_or_sac['label'] == 'e']
bulbous_cup_or_sac_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
4,e,x,y,g,t,n,f,c,b,n,...,s,p,g,p,w,o,p,k,y,d
5,e,x,f,w,t,a,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
7,e,x,f,n,t,n,f,c,b,n,...,s,p,p,p,w,o,p,n,v,d
8,e,k,f,c,f,n,f,w,n,w,...,s,w,n,p,w,o,e,w,v,l
10,e,f,y,g,t,n,f,c,b,w,...,s,g,w,p,w,o,p,n,y,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4043,e,f,y,e,t,n,f,c,b,n,...,s,p,g,p,w,o,p,n,y,d
4045,e,f,f,e,t,n,f,c,b,u,...,s,g,w,p,w,o,p,n,v,d
4047,e,x,y,g,t,n,f,c,b,n,...,s,g,g,p,w,o,p,k,v,d
4048,e,x,f,c,f,n,f,w,n,w,...,s,w,n,p,w,o,e,w,v,l


In [116]:
has_ring = training_data[training_data['ring-number'].isin(['o','t'])]

In [117]:
has_ring_p = has_ring[has_ring['label'] == 'p']
has_ring_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,y,e,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,l
2,p,f,s,n,t,p,f,c,n,p,...,s,w,w,p,w,o,p,n,s,u
3,p,x,y,g,f,f,f,c,b,g,...,k,b,n,p,w,o,l,h,y,p
14,p,x,y,n,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,v,u
16,p,k,y,n,f,s,f,c,n,b,...,k,p,p,p,w,o,e,w,v,l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4055,p,f,s,e,f,f,f,c,n,b,...,k,w,w,p,w,o,e,w,v,d
4056,p,k,s,e,f,y,f,c,n,b,...,s,w,w,p,w,o,e,w,v,p
4057,p,f,y,n,f,f,f,c,n,b,...,s,w,w,p,w,o,e,w,v,l
4058,p,x,y,y,f,f,f,c,b,h,...,k,p,n,p,w,o,l,h,y,p


In [118]:
has_ring_e = has_ring[has_ring['label'] == 'e']
has_ring_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
1,e,x,f,g,f,n,f,c,n,n,...,s,w,w,p,w,o,p,k,v,u
4,e,x,y,g,t,n,f,c,b,n,...,s,p,g,p,w,o,p,k,y,d
5,e,x,f,w,t,a,f,w,n,n,...,s,w,w,p,w,o,p,n,v,d
6,e,b,f,g,f,n,f,w,b,p,...,k,w,w,p,w,t,p,w,s,g
7,e,x,f,n,t,n,f,c,b,n,...,s,p,p,p,w,o,p,n,v,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4052,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
4053,e,f,f,w,f,n,f,w,b,p,...,s,w,w,p,w,o,e,k,a,g
4054,e,f,s,w,f,n,f,w,b,p,...,f,w,w,p,w,o,e,k,a,g
4060,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,v,l


In [119]:
thin_gill = training_data[training_data['gill-size'].isin(['n'])]

In [120]:
thin_and_white_gill = thin_gill[thin_gill['gill-color'] == 'w']

In [121]:
thin_and_white_gill_p = thin_and_white_gill[thin_and_white_gill['label'] == 'p']
thin_and_white_gill_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
14,p,x,y,n,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,v,u
73,p,x,y,n,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,s,g
299,p,b,g,w,t,n,f,w,n,w,...,s,w,w,p,w,o,p,w,c,l
405,p,f,y,y,f,n,f,c,n,w,...,y,w,y,p,w,o,e,w,v,d
418,p,x,s,w,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,v,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690,p,c,g,w,t,n,f,w,n,w,...,s,w,w,p,w,o,p,w,c,l
3739,p,f,y,n,t,p,f,c,n,w,...,s,w,w,p,w,o,p,k,s,g
3881,p,f,s,w,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,v,u
3906,p,k,y,y,f,n,f,w,n,w,...,y,y,y,p,y,o,e,w,c,l


In [122]:
thin_and_white_gill_e = thin_and_white_gill[thin_and_white_gill['label'] == 'e']
thin_and_white_gill_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
8,e,k,f,c,f,n,f,w,n,w,...,s,w,n,p,w,o,e,w,v,l
89,e,x,f,w,t,a,f,w,n,w,...,s,w,w,p,w,o,p,n,v,d
142,e,x,y,n,f,n,f,w,n,w,...,f,w,n,p,w,o,e,w,v,l
210,e,f,f,y,t,l,f,w,n,w,...,s,w,w,p,w,o,p,n,v,d
274,e,k,f,n,f,n,f,w,n,w,...,f,w,n,p,w,o,e,w,v,l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,e,f,y,c,f,n,f,w,n,w,...,f,w,n,p,w,o,e,w,v,l
3968,e,x,s,w,t,l,f,w,n,w,...,s,w,w,p,w,o,p,u,v,d
3977,e,f,f,w,t,a,f,w,n,w,...,s,w,w,p,w,o,p,u,v,d
4011,e,f,s,w,t,a,f,w,n,w,...,s,w,w,p,w,o,p,u,v,d


In [125]:
green_spore_print_color = training_data[training_data['spore-print-color'].isin(['r'])]

In [126]:
green_spore_print_color_p = green_spore_print_color[green_spore_print_color['label'] == 'p']
green_spore_print_color_p

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
182,p,b,s,p,t,n,f,c,b,g,...,s,w,w,p,w,t,p,r,v,m
345,p,b,y,p,t,n,f,c,b,g,...,s,w,w,p,w,t,p,r,v,m
466,p,b,y,p,t,n,f,c,b,r,...,s,w,w,p,w,t,p,r,v,m
554,p,b,s,w,t,n,f,c,b,r,...,s,w,w,p,w,t,p,r,v,m
665,p,b,y,w,t,n,f,c,b,g,...,s,w,w,p,w,t,p,r,v,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204,p,b,s,w,t,n,f,c,b,g,...,s,w,w,p,w,t,p,r,v,m
3435,p,f,s,w,t,n,f,c,b,g,...,s,w,w,p,w,t,p,r,v,m
3617,p,b,y,w,t,n,f,c,b,g,...,s,w,w,p,w,t,p,r,v,g
3679,p,b,y,w,t,n,f,c,b,r,...,s,w,w,p,w,t,p,r,v,m


In [127]:
green_spore_print_color_e = green_spore_print_color[green_spore_print_color['label'] == 'e']
green_spore_print_color_e

,label,cap-shape,cap-surface,cap-color,bruises,oder,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
